### Decision tree code

In [4]:
import numpy as np
from collections import Counter

In [38]:
# pip install tqdm
from tqdm.auto import tqdm

In [40]:
from tqdm.auto import tqdm

class Node:
    def __init__(self,feature_index = None, threshold = None, left = None, right = None,*,value = None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right =right
        self.value = value
    
    #to check whether it is leaf node or not
    def is_leaf_node(self):
        return self.value is not None
    
    
# create a decision tree
class Decision_tree:
    def __init__(self,min_samples_split =10, max_depth = 100, n_features= None):
        
        self.min_samples_split = min_samples_split          #specifies the minimum number of samples required 
                                                                   #to split an internal node into two branches.
        self.max_depth = max_depth                          # to intialize the max depth to how long the tree can grow
        self.n_features = n_features                        # to declare the no of features needed
        self.root = None                                    # to acces the root of the tree
    
    
    
    
    
    
    def fit(self,X,y):
        
        #this line ensures that the n_features shouldn't exceed the X.shape 
        self.n_features= X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        
        #helper function
        self.root = self._grow_tree(X,y)
        
        

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
        
        
        
        
        
    def _grow_tree(self,X,y,depth = 0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))        #gives the n labels
        
        #check the stopping criteria (provide a criteria)
        
        
        if (depth>=self.max_depth or n_labels ==1 or n_samples< self.min_samples_split):  #if condtion meets leaf value will be return
            leaf_value = self._most_common_label(y)  
            return Node(value = leaf_value)                                                #this will return the leaf value
        
        
        feat_indexs = np.random.choice(n_feats,self.n_features, replace= False) #to get the unique features while updating the group
        
        #find the best split
        best_feature,best_threshold = self._best_split(X,y,feat_indexs)
        
        #create child nodes by(accesing the _grow_tree function)
        left_idxs,right_idxs = self._split(X[:,best_feature], best_threshold)
        left =  self._grow_tree(X[left_idxs,:],y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs,:],y[right_idxs],depth+1)
        return Node(best_feature,best_threshold,left,right)
        
        
    
    
    
    
    def _best_split(self,X,y,feat_indexs):
        best_gain = -1                               # Initialize best gain to a very low value
        split_index = None                           # To store the index of the best feature for the split
        split_threshold = None                       # To store the best threshold for the split
        
        
        for feat_index in feat_indexs:                # Iterate over all feature indices
            X_column = X[:,feat_index]                # Extract the values of the current feature (column)
            thresholds= np.unique(X_column)           # Get unique values (thresholds) in the current feature
            
            for thr in thresholds:                              # Iterate over all unique thresholds (possible split points)
                #to calculate the inforamtion gain
                gain  = self._information_gain(y,X_column,thr)        # Calculate the information gain for the current feature and threshold
                                                                  # information gain using helper funtion
                
             # if the gain is better than the best found till now, upadte the best_split parameters
        
                if gain >best_gain:
                    best_gain = gain
                    split_index = feat_index
                    split_threshold = thr
                    
        return split_index,split_threshold
        
        
        
        
        
        ## INFORMATION GAIN USING ENTROPY
        
        
#     def _information_gain(self,y,X_column,threshold):
        
        
#         # parent entropy
#         parent_entropy = self._entropy(y)
                                    
#         #create children
#         left_idxs,right_idxs= self._split(X_column,threshold)
        
#         if len(left_idxs)==0 or len(right_idxs) ==0:
#             return 0
        
#         #calculate the weighted average of the entropy of the children
#         n = len(y)
#         n_left,n_right = len(left_idxs), len(right_idxs)
#         e_left,e_right = self._entropy(y[left_idxs]),self._entropy(y[right_idxs])
        
#         child_entropy = (n_left/n) * e_left / (n_right/n) * e_right   
        
#         #Calculate the information gain
        
#         information_gain = parent_entropy - child_entropy
#         return information_gain
        
     
        
        ## INFORMATION GAIN USING GINI INDEX
        
    def _information_gain(self, y, X_column, threshold):
        # Use Gini impurity 
        parent_gini = 1 - np.sum((np.bincount(y) / len(y)) ** 2)
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        left_gini = 1 - np.sum((np.bincount(y[left_idxs]) / len(left_idxs)) ** 2)
        right_gini = 1 - np.sum((np.bincount(y[right_idxs]) / len(right_idxs)) ** 2)
        child_gini = (len(left_idxs) / len(y)) * left_gini + (len(right_idxs) / len(y)) * right_gini
        return parent_gini - child_gini
  
    
    
    
    
    def _split(self,X_column,split_threshold):
        
        # Get indices of the data points that belong to the left and right node (<= threshold,>threshold)
        
        left_idxs = np.argwhere(X_column <= split_threshold).flatten()   #argwhere is used to convert the output from 2d array to 1d array
        right_idxs =np.argwhere(X_column > split_threshold).flatten()
        
        return left_idxs,right_idxs
        
        

    # Entropy formula =  -summization(p(x))*log2(p(x))
    
    def _entropy(self,y):            
        hist  = np.bincount(y)                   #it will create a histogram of number(telling the no. of times a digit occurs)
        ps  = hist/len(y)                        # formula for p(x)  = number of times class appeared / number of total nodes
        
        return -np.sum([p* np.log(p) for p in ps if p>0])   #formula for entropy with a condition
        
        
        
    def _most_common_label(self,y):                 # helper function to get the most common leaf value
        common = Counter(y)
        value = common.most_common(1)[0][0]
        return value
        
        
        
        
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    
    
    
   

### Random Forest

In [34]:
## initialzing function of  the RandomForest model
class RandomForest:
    def __init__(self,n_trees = 10,max_depth= 10, min_samples_split =2, n_features = None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []
        
    def fit(self,X,y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = Decision_tree(max_depth = self.max_depth,
                                min_samples_split = self.min_samples_split,
                                n_features = self.n_features)
            X_sample,y_sample = self._bootstrap_feature(X,y)
            tree.fit(X_sample,y_sample)
            self.trees.append(tree)
            
    # here we use most common function that will return the most common occured values using the counter
    
    def _most_common_label(self,y):                            # helper function to get the most common leaf value
        
        common = Counter(y)
        value = common.most_common(1)[0][0]
        return value
    
    def _bootstrap_feature(self,X,y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples,n_samples,replace = True) # storing the random shuffled and repeated data  into idxs #some samples will be repeated while other may be left out 
        return X[idxs],y[idxs]                                       #returning it in X and y
        
        
        
        
    def Predict(self,X):
        predictions  = np.array([tree.predict(X)for tree in self.trees])
        tree_preds = np.swapaxes(predictions,0,1)                      
        predictions = np.array([self._most_common_label(pred) for pred in tree_preds])
        return predictions
        

#### training the model

In [41]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

data  = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=45)


def accuracy(y_true,y_pred):
    accuracy = np.sum(y_true == y_pred)/len(y_true)
    return accuracy

clf = RandomForest()
clf.fit(X_train,y_train)
predcitions = clf.Predict(X_test)


acc = accuracy(y_test,predcitions)
print("the accuacy of the RandomForest model = ",acc)

KeyboardInterrupt: 